In [1]:
import folium

In [2]:
%run "../scripts/ETL.py"

22/09/21 01:13:30 WARN Utils: Your hostname, mast30034 resolves to a loopback address: 127.0.1.1; using 45.113.234.45 instead (on interface eth0)
22/09/21 01:13:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/21 01:13:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/21 01:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/21 01:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/21 01:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


22/09/21 01:14:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
final_join.head()
num_transactions_by_postcode = final_join.groupBy('postcode').count()
num_transactions_by_postcode.head()

Row(postcode='7252', count=4695)

In [4]:
num_transactions_by_postcode.head()
num_transactions_by_postcode.count()

3165

In [5]:
postcodes_data1 = spark.read.option("header", True).csv('../data/visualisations_postcodes.csv')
postcodes_data2 = postcodes_data1.withColumnRenamed('Postcode', 'postcode')
postcodes_data = postcodes_data2.dropDuplicates(['postcode'])

In [6]:
transactions_location = num_transactions_by_postcode.join(postcodes_data,['postcode'], 'inner')
transactions_location.head()


Row(postcode='2136', count=1758, Suburb='Burwood Heights', State='NSW', Lat='-33.890', Lon='151.100')

In [7]:
transactions_location.count()

3136

As we can see above - 30 postcodes in the transactions dataset were not found in the postcodes information dataset and hence their lattitude and longitude values were not retrieved

In [8]:
# conver transactions location to pandas df
transactions_location_pdf = transactions_location.toPandas()

aus_coords = [-25.2744, 133.7751]
m = folium.Map(aus_coords, tiles='OpenStreetMap', zoom_start=4.5)

for index, row in transactions_location_pdf.iterrows():
    if row['count'] >= 5000:
        marker_color = 'darkred'
        fill_color = 'darkred'
    elif row['count'] >= 1000:
        marker_color = 'red'
        fill_color = 'red'
    elif row['count'] >= 500:
        marker_color = 'lightred'
        fill_color = 'lightred'
    elif row['count'] >= 100:
        marker_color = 'orange'
        fill_color = 'orange'
    elif row['count'] <= 50 :
        marker_color = 'yellow'
        fill_color = 'yellow'
    else:
        marker_color='darkpurple'
        fill_color = 'darkpurple'
        
    folium.Circle(
          location=[row['Lat'], row['Lon']],
          popup= 'Number of transactions: ' +str(row['count']),
          tooltip=row['Suburb'],
          radius=row['count'],
          color=marker_color,
          fill=True,
          fill_color=fill_color,
       ).add_to(m)
m.save('../plots/bubble_plot_num_transactions_by_location.html')
m

NameError: name 'transactions_location_pdf' is not defined